# Modèle ARIMA en R sur tous les voxels de la matière grise 

In [1]:
import nibabel as nib
import pandas as pd
import numpy as np

import nibabel as nib

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [2]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface


In [11]:
# creation of a subworflow to calculate the arima residus
arimaResidus = pe.Workflow(name='arimaResidus')

In [12]:
sourcedir = '/scratch/user/hirsch/datadir4/data_results_py'


from nipype import SelectFiles, Node
templates = dict(gmMask=sourcedir+ "/structural/normalized_files/" + "wc1*.nii",
                 bpfile=sourcedir+ "/functionnal/bandpassedFile/" + "wcra*_merged_bp.nii.gz")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()

{'bpfile': <undefined>, 'gmMask': <undefined>}

bpfile = '/scratch/user/hirsch/datadir4/data_results_py/wcrat0009_epi_s04_d0001_merged_bp.nii.gz'
bp =nib.load(bpfile)
bparray = np.asarray(bp.dataobj).copy()

In [13]:
def computeGm(gmMask):
    
    import nibabel as nib
    import numpy as np
    import os
    
    # on regarde le grey matter
    i1=nib.load(gmMask)         
    i1array=np.asarray(i1.dataobj).copy() 
    i1array[(i1array )< 0.2] = 0
    # binary mask the resulting image
    i1array[i1array > 0] = 1
    gm_coord = np.transpose(np.nonzero(i1array))
    print len(gm_coord)
    
    out_file = os.getcwd() + '/' + 'gm_coord_file.npy'
    np.save(out_file, gm_coord)
    return out_file
    

In [14]:
# identify all the gm voxels
computeGmVoxels = Node(Function(input_names=['gmMask'],
                                output_names=['out_file'],
                                function=computeGm),
                                name='computeGmVoxels')

arimaResidus.connect(filesource, "gmMask", computeGmVoxels, "gmMask")

In [15]:
def computeArimaResidu(gmPointsFile, signalFile):
    
    import nibabel as nib
    import numpy as np
    import pandas as pd
    import os
    
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    pandas2ri.activate()

    ro.r('library(stats)')
   
    gmPoints = np.load(gmPointsFile)
    print gmPoints.size
    print gmPoints[0]
    
    bp =nib.load(signalFile) 
    bparray = np.asarray(bp.dataobj).copy()
    
    # p.full((3, 5), 7, dtype=int)
    result = np.zeros(bparray.shape,  dtype=np.float)
    
    nb_errors_arima=0
    
    for i in range(len(gmPoints)):
        
        #print gmPoints[i][0], gmPoints[i][1],gmPoints[i][2]
    
        # time serie associated to point i
        bp_ts = bparray[gmPoints[i][0], gmPoints[i][1],gmPoints[i][2], : ]
        #print bp_ts
    
        # covert numpy array to panda
        l = len(bp_ts)
        index = ['Row'+str(j) for j in range(1, l+1)]
        df = pd.DataFrame(bp_ts, index=index)
        #print df
        rdf = pandas2ri.py2ri(df)
        ro.globalenv['r_timeseries'] = rdf
        
        try:
            # model the time serie with ARIMA
            ro.r('fit <- arima(r_timeseries, order = c(1,1,1))')
            # get the residu
            residu = ro.r('fit$residuals')
            # result update
            result[gmPoints[i][0], gmPoints[i][1],gmPoints[i][2], : ] = residu
        except:
            nb_errors_arima += 1
            print "exception arima"
    
    #print residu
    print "arima errors"
    print nb_errors_arima
    
    # save the residu array
    out_file = os.getcwd() + '/' + 'residu.npy'
    np.save(out_file, result)
    
    return out_file
    
    
    
    
    

In [16]:
# identify all the gm voxels
computeArimaResiduNode = Node(Function(input_names=['gmPointsFile', 'signalFile'],
                                output_names=['out_file'],
                                function=computeArimaResidu),
                                name='computeArimaResiduNode')

arimaResidus.connect(computeGmVoxels,  'out_file', computeArimaResiduNode, "gmPointsFile")
arimaResidus.connect(filesource,  'bpfile', computeArimaResiduNode, "signalFile")

In [17]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/scratch/user/hirsch/datadir4/data_results_py'

# for motion correction  plot files
arimaResidus.connect(computeGmVoxels,  'out_file', datasink, 'functionnal.arima.gmVoxels')
arimaResidus.connect(computeArimaResiduNode,  'out_file', datasink, 'functionnal.arima.arimaResidus')

In [18]:
arimaResidus.run()

INFO:workflow:['check', 'execution', 'logging']
INFO:workflow:Running serially.
INFO:workflow:Executing node filesource in dir: /tmp/tmpTfQlK0/arimaResidus/filesource
INFO:workflow:Runtime memory and threads stats unavailable
INFO:workflow:Executing node computeGmVoxels in dir: /tmp/tmpo23zEn/arimaResidus/computeGmVoxels
INFO:workflow:Executing node computeArimaResiduNode in dir: /tmp/tmpq0geUm/arimaResidus/computeArimaResiduNode


148054


INFO:workflow:Executing node datasink in dir: /tmp/tmpP1vYN_/arimaResidus/datasink
/srv/shares/softs/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Error in solve.default(res$hessian * n.used, A) : 
  routine Lapack dgesv : le système est exactement singulier : U[1,1] = 0

  warnings.warn(x, RRuntimeWarning)
/srv/shares/softs/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: De plus : 
  warnings.warn(x, RRuntimeWarning)
/srv/shares/softs/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/srv/shares/softs/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: In arima(r_timeseries, order = c(1, 1, 1)) :
  warnings.warn(x, RRuntimeWarning)
/srv/shares/softs/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
 
  warnings.warn(x, RRuntimeWarning)
/srv

444162
[10 42 32]
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
except

exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exception arima
exceptio

# Brouillons

In [ ]:
gm_file = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/gmVoxels/gm_coord_file.npy'
bpfile = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/bandpassedFile/wcrat0009_epi_s04_d0001_merged_bp.nii.gz'


In [ ]:
computeArimaResidu(gm_file, bpfile)

In [ ]:
gm_file = '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files/wc1t0009_t1_s03.nii'
gm_pts =  computeGm(gm_file)
print gm_pts.shape
print gm_pts
    
   

In [ ]:
# bpfile = '/scratch/user/hirsch/datadir4/data_results_py/wcrat0009_epi_s04_d0001_merged_bp.nii.gz'
bp =nib.load(bpfile) 
bparray = np.asarray(bp.dataobj).copy()
ts_bp =  bparray[10, 42,  39, :]
plt.plot(ts_bp)

In [ ]:
ts_bp

In [ ]:
# covert numpy array to panda
l = len(ts_bp)
index = ['Row'+str(i) for i in range(1, l+1)]
df = pd.DataFrame(ts_bp, index=index)

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

ro.r('library(stats)')

rdf = pandas2ri.py2ri(df)
ro.globalenv['r_timeseries'] = rdf

In [ ]:
rdf

## Modélisation Arima

In [ ]:
ro.r('fit <- arima(r_timeseries)')
print(ro.r('summary(fit)'))

In [ ]:
ro.r('fit <- arima(r_timeseries, order = c(1,1,1))')
print(ro.r('summary(fit)'))


## plotting des residus ARIMA

In [ ]:
residu = ro.r('fit$residuals')

In [ ]:
rcParams['figure.figsize'] = 15, 6
plt.plot(residu)

## plotting des autocorrelations des residus ARIMA

In [ ]:
fig = plt.figure(figsize=(15,6))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(residu, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(residu, lags=40, ax=ax2)

## Comparaison des paramètres p, d, q 

In [ ]:
ro.r('AIC(fit)')

In [ ]:
ro.r('fit1 <- arima(r_timeseries, order = c(1,4,1))')

In [ ]:
ro.r('AIC(fit1)')

## qqplot

In [ ]:
import statsmodels.api as sm
fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(111)
fig = sm.qqplot(residu, line='q', ax=ax, fit=True)